In [2]:
source("utils/plot.R")

### Methods diagram

In [33]:
diagram <- ggplot() +
    draw_image(snakemake@input[["method_diagram"]]) +
    turkana_theme

### Selection strength scatterplot

In [15]:
selstrength <- read_tsv(snakemake@input[["selstrength"]], col_types=cols())

In [41]:
selstrength_scatter <- ggplot(selstrength) +
    geom_point(aes(x=true_log_selection_coefficient, y=predicted_log_selection_coefficient), size=0.5) +
    geom_abline(linetype='dashed') +
    scale_x_continuous(labels=function(x){round(10**x, 2)}, n.breaks=3) +
    scale_y_continuous(labels=function(x){round(10**x, 2)}, n.breaks=3) +
    labs(
        x = "True sel. coefficient",
        y = "Predicted sel. coefficient"
    ) +
    turkana_colour +
    turkana_theme +
    theme(
        legend.position="none",
        plot.margin = margin(18, 7, 7, 7)
    )

### Sweep mode ROC curve

In [42]:
sweepmode <- read_csv(snakemake@input[["sweepmode"]], col_types=cols()) %>%
    mutate(reference_str=paste0(sweepmode_factor_short(reference_label), ' (', scales::percent(auc), ')'))

In [43]:
sweepmode_roc <- ggplot(sweepmode) +
    geom_line(aes(x=false_positive_rate, y=true_positive_rate, colour=reference_str)) +
    geom_abline(linetype='dashed') +
    guides(colour=guide_legend(title='Reference')) +
    labs(
        x='False positive rate',
        y='True positive rate'
    ) +
    turkana_colour +
    turkana_theme +
    theme(
        legend.position=c(1, 0),
        legend.justification=c(1, 0),
        legend.background=element_rect(colour='darkgrey', size=0.5),
        legend.title=element_blank(),
        plot.margin = margin(18, 7, 7, 7)
    )

### CLUES result

In [44]:
locus_of_interest <- 23886245

clues_results <- read_tsv(snakemake@input[["clues"]], col_types=cols(locus=col_integer())) %>%
    filter(locus == locus_of_interest)

s_estimate <- clues_results$s_estimate[1]

In [45]:
clues <- ggplot(clues_results) +
        geom_raster(aes(x=gen, y=freq, fill=exp(log_prob)), hjust=0, vjust=0.5) +
        geom_text(aes(x=280, y=1.0, label=paste0('s = ', signif(s_estimate, 3))),
                  colour='white', hjust=0, vjust=1, size=5
                 ) +
        scale_x_reverse() +
        labs(
            x='Generations ago',
            y='Allele freq.'
        ) +
        turkana_theme +
        theme(
            legend.position = 'none',
            plot.subtitle=element_text(size=10, face='italic')
        )

### Empirical inferences

In [46]:
read_empirical <- function(f) {
    result <- read_tsv(f, col_types=cols()) %>%
        select(!uuid) %>%
        mutate(filename=basename(tools::file_path_sans_ext(f))) %>%
        separate(filename, into=c('target', 'dataset', NA, 'replicate'), sep="_") %>%
        separate(replicate, into=c(NA, 'replicate'), sep='-', convert=TRUE) %>%
        mutate(dataset=tools::toTitleCase(dataset))
    return(result)
}

In [47]:
empirical_files <- snakemake@input[["empirical_inferences"]]

empirical_selstrength <- bind_rows(lapply(empirical_files[str_detect(empirical_files, 'sel-strength')], read_empirical))

# Currently not using this one
empirical_sweepmode <- bind_rows(lapply(empirical_files[str_detect(empirical_files, 'sweep-mode')], read_empirical))

In [48]:
empirical <- ggplot(empirical_selstrength) +
    geom_boxplot(aes(x=dataset, y=predicted_log_selection_coefficient)) +
    geom_point(aes(x=dataset, y=predicted_log_selection_coefficient)) +
    scale_y_continuous(labels=function(x){signif(10**x, 3)}, n.breaks=3) +
    scale_x_discrete(limits=rev) +
    labs(
        y = "Predicted sel. coefficient"
    ) +
    turkana_theme +
    theme(
        axis.title.x = element_blank(),
        plot.margin = margin(7, 7, 7, 18)
    )

### Put them together

In [50]:
bottom_grid <- plot_grid(selstrength_scatter, sweepmode_roc, clues, empirical,
                         nrow=2, ncol=2, labels=c("B", "C", "D", "E"), align='hv', axis='lb')

final_fig <- plot_grid(NULL, bottom_grid, labels=c("A", NA), nrow=2, rel_heights=c(1, 2))

turkana_save(snakemake@output[["figure"]], final_fig, width=6, asp=2.5/3)